In [35]:
#import libraries
import cv2
import numpy as np
import tensorflow as tf

# Load the Haar Cascade XML file
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Load the pre-trained face detection model from TensorFlow
model = tf.keras.applications.MobileNetV2(weights='imagenet')

# Open a video capture object
video_path = 'D:/Assesment/faces01.mp4'
cap = cv2.VideoCapture(video_path)

# Get the video's frame dimensions and frames per second (fps)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define the codec and create a VideoWriter object
output_path = 'D:/Assesment/faces_detected.mp4'
fourcc = cv2.VideoWriter_fourcc(*'XVID')  # You can change the codec as needed
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# Initialize total face count
total_face_count = 0

# ...
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces using the Haar Cascade
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Count the number of detected faces
    num_faces = len(faces)
    total_face_count += num_faces

    # Display bounding boxes around detected faces
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Create a white text background for both face count and total face count
    bg_height = 50
    bg_width = 250
    bg = np.ones((bg_height, bg_width, 3), dtype=np.uint8) * 255

    # Display the number of detected faces on the white background
    text = f"Faces Detected: {num_faces}"
    text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)[0]
    text_x = bg_width - text_size[0] - 10
    text_y = text_size[1] + 10
    cv2.putText(bg, text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)

    # Display the total face count on the white background
    total_text = f"Total Faces: {total_face_count}"
    total_text_size = cv2.getTextSize(total_text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)[0]
    total_text_x = bg_width - total_text_size[0] - 10
    total_text_y = 2 * (total_text_size[1] + 10)
    cv2.putText(bg, total_text, (total_text_x, total_text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)

    # Overlay the white backgrounds with the text on the frame (upper-right corner)
    frame[10:10 + bg_height, frame.shape[1] - bg_width:frame.shape[1]] = bg

    # Write the processed frame to the output video
    out.write(frame)

    # Display the frame
    cv2.imshow('Face Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object, video writer, and close windows
cap.release()
out.release()
cv2.destroyAllWindows()
